In [200]:
import pickle
with open('../scenario-properties-elliptical.pkl', 'rb') as f:
    elliptical = pickle.load(f)

with open('../scenario-properties-not_elliptical.pkl', 'rb') as f:
    circular = pickle.load(f)

In [2]:
# # loop through all of the colliions pairs and print when species1.sym_name and species2.sym_name are the same
# for i in range(len(data.collision_pairs)):
#     if data.collision_pairs[i].species1.sym_name == data.collision_pairs[i].species2.sym_name:
#         print(data.collision_pairs[i].species1.sym_name, data.collision_pairs[i].species2.sym_name)
#         print(data.collision_pairs[i].eqs)
#         print('\n')

N_0.00141372kg N_0.00141372kg
Matrix([[-1.08678358939545e-11*pi*N_0.00141372kg_1**2, 0, 0, 0, 0, 0], [-1.07410642931825e-11*pi*N_0.00141372kg_2**2, 0, 0, 0, 0, 0], [-1.06164979782204e-11*pi*N_0.00141372kg_3**2, 0, 0, 0, 0, 0], [-1.04940860940391e-11*pi*N_0.00141372kg_4**2, 0, 0, 0, 0, 0], [-1.03737792431303e-11*pi*N_0.00141372kg_5**2, 0, 0, 0, 0, 0], [-1.02555294356653e-11*pi*N_0.00141372kg_6**2, 0, 0, 0, 0, 0], [-1.01392900416296e-11*pi*N_0.00141372kg_7**2, 0, 0, 0, 0, 0], [-1.00250157448469e-11*pi*N_0.00141372kg_8**2, 0, 0, 0, 0, 0], [-9.91266249880633e-12*pi*N_0.00141372kg_9**2, 0, 0, 0, 0, 0], [-9.8021874842115e-12*pi*N_0.00141372kg_10**2, 0, 0, 0, 0, 0]])


N_2kg N_2kg
Matrix([[7.0640933310704e-7*pi*N_2kg_1**2, -4.07543846023292e-8*pi*N_2kg_1**2, 0, 0, 0, 0], [5.55850077172196e-7*pi*N_2kg_2**2, 8.32432482721646e-8*pi*N_2kg_2**2, 1.87968625130694e-8*pi*N_2kg_2**2, 0, 0, 0], [6.90072368584324e-7*pi*N_2kg_3**2, -3.98118674183264e-8*pi*N_2kg_3**2, 0, 0, 0, 0], [6.55880380877443e-7*pi*

In [24]:
num = 95
print(elliptical.collision_pairs[num].species1.sym_name)
print(elliptical.collision_pairs[num].species2.sym_name)

N_2kg_e0.00625
N_750kg_e0.018750000000000003


In [25]:
elliptical.collision_pairs[num].eqs

Matrix([
[0, 0, 0, 0,   1.10036838426289e-7*pi*N_2kg_e0.00625_1*N_750kg_e0.018750000000000003_1, 0,   -1.22263153806988e-7*pi*N_2kg_e0.00625_1*N_750kg_e0.018750000000000003_1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   -1.22263153806988e-7*pi*N_2kg_e0.00625_1*N_750kg_e0.018750000000000003_1, 0, 0, 0, 0, 0],
[0, 0, 0, 0,   1.26878821963219e-7*pi*N_2kg_e0.00625_2*N_750kg_e0.018750000000000003_2, 0,   -1.20836973298303e-7*pi*N_2kg_e0.00625_2*N_750kg_e0.018750000000000003_2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   -1.20836973298303e-7*pi*N_2kg_e0.00625_2*N_750kg_e0.018750000000000003_2, 0, 0, 0, 0, 0],
[0, 0, 0, 0,   1.07492042029481e-7*pi*N_2kg_e0.00625_3*N_750kg_e0.018750000000000003_3, 0,   -1.19435602254979e-7*pi*N_2kg_e0.00625_3*N_750kg_e0.018750000000000003_3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   -1.19435602254979e-7*pi*N_2kg_e0.00625_3*N_750kg_e0.018750000000000003_3, 0, 0, 0, 0, 0],
[0, 0, 0, 0,   1.12155545130043e-7*pi*N_2kg_e0.00625_4*N_750kg_e0.018750000000000003_4, 0,    -1.1805846855794e-7*pi*N_2kg_e

In [38]:
# find the collision pair in elliptical where both sym_names contain B
for i in range(len(elliptical.collision_pairs)):
    if 'B' in elliptical.collision_pairs[i].species1.sym_name and 'B' in elliptical.collision_pairs[i].species2.sym_name:
        print(elliptical.collision_pairs[i].species1.sym_name, elliptical.collision_pairs[i].species2.sym_name)
        print(elliptical.collision_pairs[i].eqs)
        print('\n')

B_e0.00625 B_e0.09375
Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.92600571627296e-7*pi*B_e0.00625_1*B_e0.09375_1, -3.92600571627296e-7*pi*B_e0.00625_1*B_e0.09375_1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.88020947549898e-7*pi*B_e0.00625_2*B_e0.09375_2, -3.88020947549898e-7*pi*B_e0.00625_2*B_e0.09375_2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.83520989422369e-7*pi*B_e0.00625_3*B_e0.09375_3, -3.83520989422369e-7*pi*B_e0.00625_3*B_e0.09375_3], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.79098860106791e-7*pi*B_e0.00625_4*B_e0.09375_4, -3.79098860106791e-7*pi*B_e0.00625_4*B_e0.09375_4], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.74752775118175e-7*pi*B_e0.00625_5*B_e0.09375_5, -3.74752775118175e-7*pi*B_e0.00625_5*B_e0.09375_5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.70481000823954e-7*pi*B_e0.00625_6*B_e0.09375_6, -3.70481000823954e-7

In [67]:
# print out each name in the ellitpical scenario
for i in range(len(elliptical.collision_pairs)):
    print(elliptical.collision_pairs[i].species1.sym_name, elliptical.collision_pairs[i].species2.sym_name)

print('Length of collision pairs in elliptical:', len(elliptical.collision_pairs))

N_0.00141372kg_e0.00625 N_0.00141372kg_e0.00625
N_0.00141372kg_e0.00625 N_0.00141372kg_e0.018750000000000003
N_0.00141372kg_e0.00625 N_0.00141372kg_e0.043750000000000004
N_0.00141372kg_e0.00625 N_0.00141372kg_e0.06875
N_0.00141372kg_e0.00625 N_0.00141372kg_e0.09375
N_0.00141372kg_e0.00625 N_2kg_e0.00625
N_0.00141372kg_e0.00625 N_2kg_e0.018750000000000003
N_0.00141372kg_e0.00625 N_2kg_e0.043750000000000004
N_0.00141372kg_e0.00625 N_2kg_e0.06875
N_0.00141372kg_e0.00625 N_2kg_e0.09375
N_0.00141372kg_e0.00625 N_20kg_e0.00625
N_0.00141372kg_e0.00625 N_20kg_e0.018750000000000003
N_0.00141372kg_e0.00625 N_20kg_e0.043750000000000004
N_0.00141372kg_e0.00625 N_20kg_e0.06875
N_0.00141372kg_e0.00625 N_20kg_e0.09375
N_0.00141372kg_e0.018750000000000003 N_0.00141372kg_e0.00625
N_0.00141372kg_e0.018750000000000003 N_0.00141372kg_e0.018750000000000003
N_0.00141372kg_e0.018750000000000003 N_0.00141372kg_e0.043750000000000004
N_0.00141372kg_e0.018750000000000003 N_0.00141372kg_e0.06875
N_0.00141372kg_e0

In [78]:
# take the last collision pair from each and call it a new name
elliptical_B = elliptical.collision_pairs[-1]
circular_B = circular.collision_pairs[-1]

In [82]:
elliptical_B.collision_pair_by_shell[0].fragments

array([[[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.05, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.05, 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.05, 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.05, 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0

In [76]:
circular_B.eqs

Matrix([
[0.000238940239964483*pi*S_750kg_1**2,  2.38983711308059e-5*pi*S_750kg_1**2,   6.0859881006145e-7*pi*S_750kg_1**2, 0, -4.34713435758178e-7*pi*S_750kg_1**2, 0],
[0.000235089674184886*pi*S_750kg_2**2,  2.47259300029062e-5*pi*S_750kg_2**2,  5.58535343245491e-7*pi*S_750kg_2**2, 0, -4.29642571727301e-7*pi*S_750kg_2**2, 0],
[ 0.00022997457920421*pi*S_750kg_3**2,  2.66155604313985e-5*pi*S_750kg_3**2,  7.64387854431866e-7*pi*S_750kg_3**2, 0, -4.24659919128815e-7*pi*S_750kg_3**2, 0],
[0.000227963032220811*pi*S_750kg_4**2,  2.56895227582077e-5*pi*S_750kg_4**2,  7.34586026582737e-7*pi*S_750kg_4**2, 0, -4.19763443761564e-7*pi*S_750kg_4**2, 0],
[0.000226822683151045*pi*S_750kg_5**2,  2.40982891817918e-5*pi*S_750kg_5**2,  5.49810299885908e-7*pi*S_750kg_5**2, 0, -4.14951169725214e-7*pi*S_750kg_5**2, 0],
[0.000225488325701972*pi*S_750kg_6**2,  2.26852311116916e-5*pi*S_750kg_6**2,  4.30732236297941e-7*pi*S_750kg_6**2, 0, -4.10221177426611e-7*pi*S_750kg_6**2, 0],
[0.000222232959132438*pi*S_750k

In [53]:
circular.species_names

['N_0.00141372kg', 'N_2kg', 'N_20kg', 'S_148kg', 'S_750kg', 'B']

In [48]:
circular_B.eqs

Matrix([
[ 0.000820731494986862*pi*B_1**2,  8.77265977301193e-5*pi*B_1**2,  2.94450428720472e-6*pi*B_1**2, 0, 0,  -7.85201143254592e-7*pi*B_1**2],
[ 0.000812283051600956*pi*B_2**2,  8.64704681614947e-5*pi*B_2**2,  2.01770892725947e-6*pi*B_2**2, 0, 0,  -7.76041895099795e-7*pi*B_2**2],
[ 0.000807848612119277*pi*B_3**2,  8.04243514818707e-5*pi*B_3**2,  2.05183729340967e-6*pi*B_3**2, 0, 0,  -7.67041978844737e-7*pi*B_3**2],
[ 0.000790951861726809*pi*B_4**2,  8.65293148193751e-5*pi*B_4**2,  2.57787224872618e-6*pi*B_4**2, 0, 0,  -7.58197720213582e-7*pi*B_4**2],
[ 0.000785182014427601*pi*B_5**2,  8.28765762173845e-5*pi*B_5**2,  1.91123915310269e-6*pi*B_5**2, 0, 0,  -7.49505550236351e-7*pi*B_5**2],
[  0.00077625031697639*pi*B_6**2,  8.16169644815171e-5*pi*B_6**2,  2.18583790486133e-6*pi*B_6**2, 0, 0,  -7.40962001647909e-7*pi*B_6**2],
[ 0.000768862237033769*pi*B_7**2,  7.91718224643177e-5*pi*B_7**2,  2.27094748683215e-6*pi*B_7**2, 0, 0,  -7.32563705429727e-7*pi*B_7**2],
[ 0.000760323572330901*pi

In [104]:
for i in elliptical.species['debris']:
    print(i.sym_name)

N_0.00141372kg_e0.00625
N_0.00141372kg_e0.018750000000000003
N_0.00141372kg_e0.043750000000000004
N_0.00141372kg_e0.06875
N_0.00141372kg_e0.09375
N_2kg_e0.00625
N_2kg_e0.018750000000000003
N_2kg_e0.043750000000000004
N_2kg_e0.06875
N_2kg_e0.09375
N_20kg_e0.00625
N_20kg_e0.018750000000000003
N_20kg_e0.043750000000000004
N_20kg_e0.06875
N_20kg_e0.09375
N_750kg_e0.00625
N_750kg_e0.018750000000000003
N_750kg_e0.043750000000000004
N_750kg_e0.06875
N_750kg_e0.09375


In [105]:
import numpy as np

debris_species = elliptical.species['debris']
n_shells = elliptical.n_shells
n_mass_bins = 4  # Assuming there are 4 mass bins
n_eccentricity_bins = len(debris_species[0].eccentricity_bins)
n_debris_species = n_mass_bins * n_eccentricity_bins

all_gammas = []

for species_pair in elliptical.collision_pairs:
    for elliptical_pair in species_pair.collision_processed:
        gamma_matrix_for_ellipitical_pair = np.zeros((n_shells, n_debris_species))
        fragments = elliptical_pair.fragments

        if fragments is not None:
            for shell_index in range(n_shells):
                for species_index in range(n_mass_bins):
                    for eccentricity_index in range(n_eccentricity_bins):
                        try:
                            frags = fragments[shell_index, species_index, eccentricity_index]

                            if frags is not None:
                                debris_index = species_index * n_eccentricity_bins + eccentricity_index
                                gamma_matrix_for_ellipitical_pair[shell_index, debris_index] += frags
                        except Exception as e:
                            print(f"Error accessing fragments: {e}")
                            print(f"shell_index: {shell_index}, species_index: {species_index}, eccentricity_index: {eccentricity_index}")

        all_gammas.append(gamma_matrix_for_ellipitical_pair)

In [134]:
def is_catastrophic(mass1, mass2, vels):
    """
    Determines if a collision is catastropic or non-catastrophic by calculating the 
    relative kinetic energy. If the energy is greater than 40 J/g, the collision is
    catastrophic from Johnson et al. 2001 (Collision Section)

    Args:
        mass1 (float): mass of species 1, kg
        mass2 (float): mass of species 2, kg
        vels (np.ndarray): array of the relative velocities (km/s) for each shell
    
    Returns:
        shell-wise list of bools (true if catastrophic, false if not catastrophic)
    """

    if mass1 <= mass2:
        smaller_mass = mass1
    else:
        smaller_mass = mass2
    
    smaller_mass_g = smaller_mass * (1000) # kg to g
    energy = [0.5 * smaller_mass * (v)**2 for v in vels] # Need to also convert km/s to m/s
    is_catastrophic = [True if e/smaller_mass_g > 40 else False for e in energy]

    return is_catastrophic

In [135]:
from sympy import symbols, Matrix, pi, S, Expr, zeros

collision_pair = elliptical.collision_pairs[-1]
scen_properties = elliptical
print(collision_pair.collision_pair_by_shell[0].species1.sym_name)


# so we now know the number of fragments that are produced by each species and where they are distributed not only in the shells
# but also in the mass and eccentricity bins. This is a good start. 

# We now need to make an overall gamma matrix that replaces the values with the syms that that generated that bit of debris

collision_pair.eqs = Matrix(scen_properties.n_shells, scen_properties.species_length, lambda i, j: 0)
species1, species2 = collision_pair.species1, collision_pair.species2
meter_to_km = 1/1000

collision_pair.sigma = (species1.radius * meter_to_km + species2.radius * meter_to_km) ** 2

collision_pair.phi = pi * scen_properties.v_imp2 / (scen_properties.V * meter_to_km**3) * collision_pair.sigma * S(86400) * S(365.25)

catastrophic = is_catastrophic(species1.mass, species2.mass, scen_properties.v_imp2)

nf = zeros(len(scen_properties.v_imp2), 1)


for i, dv in enumerate(scen_properties.v_imp2):
    if catastrophic[i]:
        # number of fragments generated during a catastrophic collision (NASA standard break-up model). M is the sum of the mass of the objects colliding in kg
        n_f_catastrophic = 0.1 * scen_properties.LC**(-S(1.71)) * (species1.mass + species2.mass)**(S(0.75))
        nf[i] = n_f_catastrophic
    else:
        # number of fragments generated during a non-catastrophic collision (improved NASA standard break-up model: takes into account the kinetic energy). M is the mass of the less massive object colliding in kg
        n_f_damaging = 0.1 * scen_properties.LC**(-S(1.71)) * (min(species1.mass, species2.mass) * dv**2)**(S(0.75))
        nf[i] = n_f_damaging

collision_pair.nf = nf.transpose()


if isinstance(collision_pair.phi, (int, float, Expr)):
        phi_matrix = Matrix([collision_pair.phi] * len(collision_pair.gamma))
else:
    phi_matrix = Matrix(collision_pair.phi)

B_e0.09375


In [147]:
collision_pair.collision_pair_by_shell

In [150]:


from sympy import symbols, Matrix, pi, S, Expr, zeros
import numpy as np

# Assuming 'elliptical' is defined and contains the collision pairs and species information
collision_pair = elliptical.collision_pairs[-1]
scen_properties = elliptical
print(f"Species involved in collision: {collision_pair.species1.sym_name} and {collision_pair.species2.sym_name}")

species1, species2 = collision_pair.species1, collision_pair.species2
meter_to_km = 1/1000

collision_pair.sigma = (species1.radius * meter_to_km + species2.radius * meter_to_km) ** 2

collision_pair.phi = pi * scen_properties.v_imp2 / (scen_properties.V * meter_to_km**3) * collision_pair.sigma * S(86400) * S(365.25)

# Determine if the collision is catastrophic for each shell
catastrophic = is_catastrophic(species1.mass, species2.mass, scen_properties.v_imp2)

# Compute the number of fragments (nf) for each shell
nf = zeros(len(scen_properties.v_imp2), 1)

for i, dv in enumerate(scen_properties.v_imp2):
    if catastrophic[i]:
        # Catastrophic collision
        n_f_catastrophic = 0.1 * scen_properties.LC**(-S(1.71)) * (species1.mass + species2.mass)**(S(0.75))
        nf[i] = n_f_catastrophic
    else:
        # Non-catastrophic collision
        n_f_damaging = 0.1 * scen_properties.LC**(-S(1.71)) * (min(species1.mass, species2.mass) * dv**2)**(S(0.75))
        nf[i] = n_f_damaging

collision_pair.nf = nf.transpose()

if isinstance(collision_pair.phi, (int, float, Expr)):
    phi_matrix = Matrix([collision_pair.phi] * len(collision_pair.nf))
else:
    phi_matrix = Matrix(collision_pair.phi)

# Now process one EllipticalCollisionPair from collision_pair.collision_pair_by_shell
elliptical_pair = collision_pair.collision_pair_by_shell[0]  # Process the first elliptical pair

# Ensure that fragments are available
fragments = elliptical_pair.fragments  # Access fragments from elliptical_pair

if fragments is not None:
    # Define parameters
    n_shells = scen_properties.n_shells
    debris_species = scen_properties.species['debris']
    n_mass_bins = len(debris_species)  # Number of mass bins
    n_eccentricity_bins = len(debris_species[0].eccentricity_bins)  # Assuming all debris species have the same number of eccentricity bins
    n_debris_species = n_mass_bins * n_eccentricity_bins

    # Include 2 columns for colliding species
    total_gamma_columns = 2 + n_debris_species  # 2 for species1 and species2, rest for debris species

    # Initialize gamma for the elliptical_pair if not already done
    if not hasattr(elliptical_pair, 'gamma') or elliptical_pair.gamma is None:
        elliptical_pair.gamma = np.zeros((n_shells, total_gamma_columns))

    # Set gamma entries for colliding species to -1
    elliptical_pair.gamma[:, 0] = -1  # For species1
    elliptical_pair.gamma[:, 1] = -1  # For species2

    # Compute elliptical_pair.gamma based on fragments
    for shell_index in range(n_shells):
        for species_index in range(n_mass_bins):
            for eccentricity_index in range(n_eccentricity_bins):
                try:
                    frags = fragments[shell_index, species_index, eccentricity_index]

                    if frags is not None:
                        debris_index = species_index * n_eccentricity_bins + eccentricity_index
                        gamma_col_idx = 2 + debris_index  # Adjust index for gamma
                        elliptical_pair.gamma[shell_index, gamma_col_idx] += frags
                except Exception as e:
                    print(f"Error accessing fragments: {e}")
                    print(f"shell_index: {shell_index}, species_index: {species_index}, eccentricity_index: {eccentricity_index}")
else:
    print("No fragments found for elliptical_pair.fragments")
    # You may want to handle this case appropriately

# Now construct symbolic equations for this elliptical_pair
# Get the list of all species (including debris species)
all_species = [species for category in scen_properties.species.values() for species in category]
species_names = [species.sym_name for species in all_species]
n_species = len(species_names)

# Create symbolic variables for species populations in each shell
species_sym_vars = {}
for name in species_names:
    species_sym_vars[name] = []
    for shell_index in range(n_shells):
        sym_var = symbols(f'{name}_{shell_index + 1}')  # Shells are 1-indexed
        species_sym_vars[name].append(sym_var)

# Initialize the equations matrix
equations = Matrix.zeros(n_shells, n_species)

# Build gamma_species_names, which correspond to the columns in gamma
# First two are the colliding species
gamma_species_names = [species1.sym_name, species2.sym_name]

# Get list of debris species
debris_species_list = [species for category in scen_properties.species.values() for species in category if species.sym_name.startswith('N_') or species.sym_name.startswith('B_')]
debris_species_names = [spec.sym_name for spec in debris_species_list]
n_debris_species = len(debris_species_names)

# Include 2 columns for colliding species
total_gamma_columns = 2 + n_debris_species  # 2 for species1 and species2, rest for debris species

# Initialize gamma for the elliptical_pair
elliptical_pair.gamma = np.zeros((n_shells, total_gamma_columns))

# Set gamma entries for colliding species to -1
elliptical_pair.gamma[:, 0] = -1  # For species1
elliptical_pair.gamma[:, 1] = -1  # For species2

# Compute elliptical_pair.gamma based on fragments
for shell_index in range(n_shells):
    for species_index in range(n_mass_bins):
        for eccentricity_index in range(n_eccentricity_bins):
            try:
                frags = fragments[shell_index, species_index, eccentricity_index]

                if frags is not None:
                    debris_index = species_index * n_eccentricity_bins + eccentricity_index
                    gamma_col_idx = 2 + debris_index  # Adjust index for gamma
                    elliptical_pair.gamma[shell_index, gamma_col_idx] += frags
            except Exception as e:
                print(f"Error accessing fragments: {e}")
                print(f"shell_index: {shell_index}, species_index: {species_index}, eccentricity_index: {eccentricity_index}")

# Build gamma_species_names
gamma_species_names = [species1.sym_name, species2.sym_name] + debris_species_names
print(f"Total gamma columns: {total_gamma_columns}")
print(f"Length of gamma_species_names: {len(gamma_species_names)}")

# Map gamma columns to species indices
gamma_to_species_idx = [species_names.index(name) for name in gamma_species_names]

# Confirm lengths match
assert total_gamma_columns == len(gamma_species_names) == len(gamma_to_species_idx), "Mismatch in gamma columns and species indices"

# Process the shell corresponding to the elliptical_pair
s = elliptical_pair.shell_index  # The shell index for this collision

N_species1_s = species_sym_vars[species1.sym_name][s]
N_species2_s = species_sym_vars[species2.sym_name][s]
phi_s = collision_pair.phi[s]
nf_s = collision_pair.nf[s]

# Loop over gamma columns
for gamma_col_idx, species_idx in enumerate(gamma_to_species_idx):
    species_name = species_names[species_idx]
    gamma_s_d = elliptical_pair.gamma[s, gamma_col_idx]

    delta = phi_s * N_species1_s * N_species2_s * gamma_s_d
    if species_name in [species1.sym_name, species2.sym_name]:
        # For colliding species, gamma_s_d is -1
        equations[s, species_idx] += delta  # delta is negative
    else:
        # For debris species
        delta *= nf_s  # Multiply by number of fragments
        equations[s, species_idx] += delta

# Optional: Print the symbolic equations for verification
print("Symbolic Equations for the elliptical collision pair:")
for idx, species_name in enumerate(species_names):
    eq = equations[s, idx]
    if eq != 0:
        print(f"Shell {s + 1}, Species {species_name}: dN/dt = {eq}")


Species involved in collision: B_e0.09375 and B_e0.09375
Error accessing fragments: index 20 is out of bounds for axis 1 with size 20
shell_index: 0, species_index: 3, eccentricity_index: 3
Error accessing fragments: index 21 is out of bounds for axis 1 with size 20
shell_index: 0, species_index: 3, eccentricity_index: 4
Error accessing fragments: index 4 is out of bounds for axis 1 with size 4
shell_index: 0, species_index: 4, eccentricity_index: 0
Error accessing fragments: index 4 is out of bounds for axis 1 with size 4
shell_index: 0, species_index: 4, eccentricity_index: 1
Error accessing fragments: index 4 is out of bounds for axis 1 with size 4
shell_index: 0, species_index: 4, eccentricity_index: 2
Error accessing fragments: index 4 is out of bounds for axis 1 with size 4
shell_index: 0, species_index: 4, eccentricity_index: 3
Error accessing fragments: index 4 is out of bounds for axis 1 with size 4
shell_index: 0, species_index: 4, eccentricity_index: 4
Error accessing fragme

In [238]:
from sympy import symbols

all_symbolic_vars = scen_properties.all_symbolic_vars
# Define mass bins and eccentricity bins
mass_bins = ['0.00141372kg', '2kg', '20kg', '750kg']  # 4 mass bins
ecc_bins = ['0.00625', '0.01875', '0.04375', '0.06875', '0.09375']  # 5 ecc bins

# Initialize debris species lists
debris_species_names = []
debris_symbolic_vars = []

for s in range(n_shells):
    for mass_value in mass_bins:
        for ecc_value in ecc_bins:
            debris_species_name = f'N_{mass_value}_e{ecc_value}_{s + 1}'
            debris_species_names.append(debris_species_name)
            debris_var = symbols(debris_species_name)
            debris_symbolic_vars.append(debris_var)

# Ensure that debris species variables are included in 'all_symbolic_vars'
# If they are not included, we can append them
for debris_var in debris_symbolic_vars:
    if debris_var not in all_symbolic_vars:
        all_symbolic_vars.append(debris_var)

# Rebuild the mappings to include the debris species
species_name_to_idx = {str(var): idx for idx, var in enumerate(all_symbolic_vars)}
species_sym_vars = {str(var): var for var in all_symbolic_vars}
debris_length = len(mass_bins) * len(ecc_bins)  # Should be 20
equations_debris = Matrix.zeros(n_shells, debris_length)

# Map debris species variable names to indices in the debris equations matrix
debris_species_name_to_idx = {}
for idx, name in enumerate(debris_species_names):
    # Since debris_species_names are ordered per shell, we can calculate shell index and debris index
    s = idx // debris_length  # Integer division to get shell index
    debris_idx = idx % debris_length  # Remainder to get index within the shell
    debris_species_name_to_idx[name] = (s, debris_idx)

# Initialize the debris equations matrix
equations_debris = Matrix.zeros(n_shells, debris_length)

collision_pair = elliptical.collision_pairs[-1]
    # Map out the major components
species1 = collision_pair.species1
species2 = collision_pair.species2
species1_name = species1.sym_name
species2_name = species2.sym_name

# Conversion factor
meter_to_km = 1 / 1000

# Compute collision cross-section (sigma)
collision_pair.sigma = (species1.radius * meter_to_km + species2.radius * meter_to_km) ** 2

# Compute collision rate (phi) for each shell
collision_pair.phi = (
    pi * scen_properties.v_imp2
    / (scen_properties.V * meter_to_km**3)
    * collision_pair.sigma
    * S(86400)
    * S(365.25)
)

# Determine if the collision is catastrophic for each shell
catastrophic = is_catastrophic(species1.mass, species2.mass, scen_properties.v_imp2)

# Compute the number of fragments (nf) for each shell
nf = zeros(len(scen_properties.v_imp2), 1)
for i, dv in enumerate(scen_properties.v_imp2):
    if catastrophic[i]:
        n_f_catastrophic = 0.1 * scen_properties.LC ** (-S(1.71)) * (
            species1.mass + species2.mass
        ) ** (S(0.75))
        nf[i] = n_f_catastrophic
    else:
        n_f_damaging = 0.1 * scen_properties.LC ** (-S(1.71)) * (
            min(species1.mass, species2.mass) * dv ** 2
        ) ** (S(0.75))
        nf[i] = n_f_damaging
collision_pair.nf = nf.transpose()

# Now loop over each of the EllipticalCollisionPairs
for elliptical_pair in collision_pair.collision_pair_by_shell:
    s = elliptical_pair.shell_index  # Shell index (0-based)

    # Get species variable names including shell number
    species1_var_name = f'{species1_name}_{s + 1}'
    species2_var_name = f'{species2_name}_{s + 1}'

    # Get symbolic variables
    N_species1_s = species_sym_vars.get(species1_var_name)
    N_species2_s = species_sym_vars.get(species2_var_name)

    print(f"Species variables: {species1_var_name}, {species2_var_name}")

    if N_species1_s is None or N_species2_s is None:
        print(f"Species variables not found in all_symbolic_vars: {species1_var_name}, {species2_var_name}")
        continue  # Skip if species variables are not found

    phi_s = collision_pair.phi[s]
    nf_s = collision_pair.nf[s]

    # Process fragments from the collision
    fragments = elliptical_pair.fragments  # Should be an array of shape [n_shells, n_mass_bins, n_ecc_bins]

    if fragments is not None:
        fragments_s = fragments[s]  # Get fragments for the current shell
        n_mass_bins, n_ecc_bins = fragments_s.shape

        for mass_bin_index in range(n_mass_bins):
            for ecc_bin_index in range(n_ecc_bins):
                num_frags = fragments_s[mass_bin_index, ecc_bin_index]
                if num_frags != 0:
                    mass_value = mass_bins[mass_bin_index]
                    ecc_value = ecc_bins[ecc_bin_index]
                    # Generate debris species variable name including shell number
                    debris_species_name = f'N_{mass_value}_e{ecc_value}_{s + 1}'
                    idx_shell, idx_debris = debris_species_name_to_idx.get(debris_species_name, (None, None))
                    if idx_shell is not None and idx_debris is not None:
                        debris_var = species_sym_vars[debris_species_name]
                        # Compute delta gain for debris species (symbolic)
                        delta_gain = phi_s * N_species1_s * N_species2_s * nf_s * num_frags
                        equations_debris[s, idx_debris] += delta_gain
                    else:
                        print(f"Variable {debris_species_name} not found in debris species mapping.")
    else:
        continue  # If fragments are None, skip to the next elliptical_pair

bigger_matrix = Matrix.zeros(n_shells, debris_length)

bigger_matrix += equations_debris



Species variables: B_e0.09375_1, B_e0.09375_1
3.92600571627296e-7*pi 44517.9525875460
Species variables: B_e0.09375_2, B_e0.09375_2
3.88020947549898e-7*pi 44517.9525875460
Species variables: B_e0.09375_3, B_e0.09375_3
3.83520989422369e-7*pi 44517.9525875460
Species variables: B_e0.09375_4, B_e0.09375_4
3.79098860106791e-7*pi 44517.9525875460
Species variables: B_e0.09375_5, B_e0.09375_5
3.74752775118175e-7*pi 44517.9525875460
Species variables: B_e0.09375_6, B_e0.09375_6
3.70481000823954e-7*pi 44517.9525875460
Species variables: B_e0.09375_7, B_e0.09375_7
3.66281852714863e-7*pi 44517.9525875460
Species variables: B_e0.09375_8, B_e0.09375_8
3.62153693744029e-7*pi 44517.9525875460
Species variables: B_e0.09375_9, B_e0.09375_9
3.58094932731244e-7*pi 44517.9525875460
Species variables: B_e0.09375_10, B_e0.09375_10
3.54104022829431e-7*pi 44517.9525875460
Species variables: B_e0.09375_1, B_e0.09375_1
3.92600571627296e-7*pi 44517.9525875460
Species variables: B_e0.09375_2, B_e0.09375_2
3.8802

In [299]:
import numpy as np
from sympy import symbols, Matrix, pi, S
import tqdm

# Assume 'elliptical' is your main scenario object
scen_properties = elliptical
n_shells = scen_properties.n_shells

# Define mass bins and eccentricity bins
mass_bins = ['0.00141372kg', '2kg', '20kg', '750kg']  # 4 mass bins
ecc_bins = ['0.00625', '0.01875', '0.04375', '0.06875', '0.09375']  # 5 ecc bins

# Initialize debris species lists
debris_species_names = []
debris_symbolic_vars = []

for s in range(n_shells):
    for mass_value in mass_bins:
        for ecc_value in ecc_bins:
            debris_species_name = f'N_{mass_value}_e{ecc_value}_{s + 1}'
            debris_species_names.append(debris_species_name)
            debris_var = symbols(debris_species_name)
            debris_symbolic_vars.append(debris_var)

# Ensure that debris species variables are included in 'all_symbolic_vars'
all_symbolic_vars = scen_properties.all_symbolic_vars

for debris_var in debris_symbolic_vars:
    if debris_var not in all_symbolic_vars:
        all_symbolic_vars.append(debris_var)

# Rebuild the mappings to include the debris species
species_name_to_idx = {str(var): idx for idx, var in enumerate(all_symbolic_vars)}
species_sym_vars = {str(var): var for var in all_symbolic_vars}

# Initialize the equations matrix
n_species = len(all_symbolic_vars)
equations = Matrix.zeros(n_shells, n_species)

# Process only one collision_pair
# for collision_pair in tqdm.tqdm(elliptical.collision_pairs):
    # Map out the major components

collision_pair  = elliptical.collision_pairs[-1]

species1 = collision_pair.species1
species2 = collision_pair.species2
species1_name = species1.sym_name
species2_name = species2.sym_name

# Conversion factor
meter_to_km = 1 / 1000

# Compute collision cross-section (sigma)
collision_pair.sigma = (species1.radius * meter_to_km + species2.radius * meter_to_km) ** 2

# Compute collision rate (phi) for each shell
collision_pair.phi = (
    pi * scen_properties.v_imp2
    / (scen_properties.V * meter_to_km**3)
    * collision_pair.sigma
    * S(86400)
    * S(365.25)
)

# Determine if the collision is catastrophic for each shell
# 'is_catastrophic' should be defined elsewhere in your code
catastrophic = is_catastrophic(species1.mass, species2.mass, scen_properties.v_imp2)

# Compute the number of fragments (nf) for each shell
nf = np.zeros(len(scen_properties.v_imp2), dtype=object)
for i, dv in enumerate(scen_properties.v_imp2):
    if catastrophic[i]:
        n_f_catastrophic = 0.1 * scen_properties.LC ** (-S(1.71)) * (
            species1.mass + species2.mass
        ) ** (S(0.75))
        nf[i] = n_f_catastrophic
    else:
        n_f_damaging = 0.1 * scen_properties.LC ** (-S(1.71)) * (
            min(species1.mass, species2.mass) * dv ** 2
        ) ** (S(0.75))
        nf[i] = n_f_damaging
collision_pair.nf = nf  # 'nf' is now a NumPy array

# Now loop over each of the EllipticalCollisionPairs in this collision_pair
for elliptical_pair in collision_pair.collision_pair_by_shell:
    s_source = elliptical_pair.shell_index  # Source shell index (0-based)

    # Get species variable names including shell number
    species1_var_name = f'{species1_name}_{s_source + 1}'
    species2_var_name = f'{species2_name}_{s_source + 1}'

    # Get symbolic variables
    N_species1_s = species_sym_vars.get(species1_var_name)
    N_species2_s = species_sym_vars.get(species2_var_name)

    if N_species1_s is None or N_species2_s is None:
        continue  # Skip if species variables are not found

    phi_s = collision_pair.phi[s_source]
    nf_s = collision_pair.nf[s_source]

    # Process fragments from the collision
    fragments = elliptical_pair.fragments  # Should be an array of shape [n_destination_shells, n_mass_bins, n_ecc_bins]

    if fragments is not None:
        n_destination_shells, n_mass_bins, n_ecc_bins = fragments.shape

        for s_destination in range(n_destination_shells):
            fragments_sd = fragments[s_destination]  # Fragments ending up in shell s_destination
            for mass_bin_index in range(n_mass_bins):
                for ecc_bin_index in range(n_ecc_bins):
                    num_frags = fragments_sd[mass_bin_index, ecc_bin_index]
                    if num_frags != 0:
                        mass_value = mass_bins[mass_bin_index]
                        ecc_value = ecc_bins[ecc_bin_index]
                        # Generate debris species variable name including destination shell number
                        debris_species_name = f'N_{mass_value}_e{ecc_value}_{s_destination + 1}'
                        idx_debris = species_name_to_idx.get(debris_species_name)
                        if idx_debris is not None:
                            debris_var = species_sym_vars[debris_species_name]
                            # Compute delta gain for debris species (symbolic)
                            delta_gain = phi_s * N_species1_s * N_species2_s * nf_s * num_frags
                            equations[s_destination, idx_debris] += delta_gain
    else:
        continue  # If fragments are None, skip to the next elliptical_pair

# Now, extract the equations for the debris species and store them in a (10, 20) matrix
# Initialize the debris equations matrix
debris_length = len(mass_bins) * len(ecc_bins)  # Should be 20
equations_debris = Matrix.zeros(n_shells, debris_length)

# Map debris species variable names to indices within the shell
debris_species_idx_within_shell = {}
for idx_within_shell, debris_species_name in enumerate(debris_species_names[:debris_length]):
    base_name = '_'.join(debris_species_name.split('_')[:-1])
    debris_species_idx_within_shell[base_name] = idx_within_shell

# Populate the debris equations matrix
for s in range(n_shells):
    for mass_value in mass_bins:
        for ecc_value in ecc_bins:
            debris_species_name = f'N_{mass_value}_e{ecc_value}_{s + 1}'
            base_name = f'N_{mass_value}_e{ecc_value}'
            idx_species = species_name_to_idx.get(debris_species_name)
            idx_debris = debris_species_idx_within_shell.get(base_name)
            if idx_species is not None and idx_debris is not None:
                eq = equations[s, idx_species]
                equations_debris[s, idx_debris] += eq

# print(collision_pair.gamma)
# print(species1.sym_name, species2.sym_name)
# find index of syn_name in species_names
species_names = scen_properties.species_names
species1_idx = species_names.index(species1.sym_name)
species2_idx = species_names.index(species2.sym_name)
# print(species1_idx, species2_idx)
# find the start of the debris species index, which will be the first item in species_names that starts with 'N_'
debris_start_idx = next(i for i, name in enumerate(species_names) if name.startswith('N_'))
# print(debris_start_idx)

eq_s1 = collision_pair.gamma[:, 1].multiply_elementwise(phi_matrix).multiply_elementwise(species1.sym).multiply_elementwise(species2.sym)
eq_s2 = collision_pair.gamma[:, 1].multiply_elementwise(phi_matrix).multiply_elementwise(species1.sym).multiply_elementwise(species2.sym)

eqs = Matrix(zeros(n_shells, scen_properties.species_length))

# # add in eq_1 at species1_idx and eq_2 at species2_idx
eqs[:, species1_idx] = eq_s1
eqs[:, species2_idx] += eq_s2


    # Loop through each debris species
for i in range(len(scen_properties.species['debris'])):
    # Calculate the corresponding index in the overall species list
    deb_index = debris_start_idx + i
    # Assign the columns from equations_debris to the appropriate columns in eqs
    eqs[:, deb_index] = equations_debris[:, i]

Matrix([[-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1]])
B_e0.09375 B_e0.09375
22 22
1
Debris species index in debris list: 0
Debris species index in overall species list: 1
Debris species index in debris list: 1
Debris species index in overall species list: 2
Debris species index in debris list: 2
Debris species index in overall species list: 3
Debris species index in debris list: 3
Debris species index in overall species list: 4
Debris species index in debris list: 4
Debris species index in overall species list: 5
Debris species index in debris list: 5
Debris species index in overall species list: 6
Debris species index in debris list: 6
Debris species index in overall species list: 7
Debris species index in debris list: 7
Debris species index in overall species list: 8
Debris species index in debris list: 8
Debris species index in overall species list: 9
Debris species index in debris list: 9
Debris species index in overall species

Matrix([
[0, 0, 0, 0, 0, 0,                                                                                                                                                                      0.000843835254011342*pi*B_e0.09375_4**2, 0.00086369490730005*pi*B_e0.09375_2**2 + 0.000853678461171686*pi*B_e0.09375_3**2 + 0.00164930556292674*pi*B_e0.09375_6**2 + 0.00244591772292582*pi*B_e0.09375_7**2,                                                                                   0.000806117048375066*pi*B_e0.09375_8**2, 0.000788199304968995*pi*B_e0.09375_10**2, 0,                                                                                                                                                                                                                                                                                                                                       0.00421917627005671*pi*B_e0.09375_4**2 + 0.00250248394121433*pi*B_e0.09375_5**2,                            

In [267]:
import numpy as np
from sympy import symbols, Matrix, pi, S

# Assume 'elliptical' is your main scenario object
scen_properties = elliptical
n_shells = scen_properties.n_shells

# Define mass bins and eccentricity bins
mass_bins = ['0.00141372kg', '2kg', '20kg', '750kg']  # 4 mass bins
ecc_bins = ['0.00625', '0.01875', '0.04375', '0.06875', '0.09375']  # 5 ecc bins

# Initialize debris species lists
debris_species_names = []
debris_symbolic_vars = []

for s in range(n_shells):
    for mass_value in mass_bins:
        for ecc_value in ecc_bins:
            debris_species_name = f'N_{mass_value}_e{ecc_value}_{s + 1}'
            debris_species_names.append(debris_species_name)
            debris_var = symbols(debris_species_name)
            debris_symbolic_vars.append(debris_var)

# Ensure that debris species variables are included in 'all_symbolic_vars'
all_symbolic_vars = scen_properties.all_symbolic_vars.copy()

for debris_var in debris_symbolic_vars:
    if debris_var not in all_symbolic_vars:
        all_symbolic_vars.append(debris_var)

# Rebuild the mappings to include the debris species
species_name_to_idx = {str(var): idx for idx, var in enumerate(all_symbolic_vars)}
species_sym_vars = {str(var): var for var in all_symbolic_vars}

# Initialize the 'eqs' matrix
n_species = len(all_symbolic_vars)
eqs = Matrix.zeros(n_shells, n_species)

# Map species names to indices in 'eqs'
species_name_to_eqs_idx = {str(var): idx for idx, var in enumerate(all_symbolic_vars)}

# Process only one collision_pair
collision_pair = scen_properties.collision_pairs[-1]

# Map out the major components
species1 = collision_pair.species1
species2 = collision_pair.species2
species1_name = species1.sym_name
species2_name = species2.sym_name

# Conversion factor
meter_to_km = 1 / 1000

# Compute collision cross-section (sigma)
collision_pair.sigma = (species1.radius * meter_to_km + species2.radius * meter_to_km) ** 2

# Compute collision rate (phi) for each shell
collision_pair.phi = (
    pi * scen_properties.v_imp2
    / (scen_properties.V * meter_to_km**3)
    * collision_pair.sigma
    * S(86400)
    * S(365.25)
)

# Determine if the collision is catastrophic for each shell
# 'is_catastrophic' should be defined elsewhere in your code
catastrophic = is_catastrophic(species1.mass, species2.mass, scen_properties.v_imp2)

# Compute the number of fragments (nf) for each shell
nf = np.zeros(len(scen_properties.v_imp2), dtype=object)
for i, dv in enumerate(scen_properties.v_imp2):
    if catastrophic[i]:
        n_f_catastrophic = 0.1 * scen_properties.LC ** (-S(1.71)) * (
            species1.mass + species2.mass
        ) ** (S(0.75))
        nf[i] = n_f_catastrophic
    else:
        n_f_damaging = 0.1 * scen_properties.LC ** (-S(1.71)) * (
            min(species1.mass, species2.mass) * dv ** 2
        ) ** (S(0.75))
        nf[i] = n_f_damaging
collision_pair.nf = nf  # 'nf' is now a NumPy array

# Initialize gamma array (full of -1 for each shell)
gamma = Matrix([-1] * n_shells)

# Now loop over each of the EllipticalCollisionPairs in this collision_pair
for elliptical_pair in collision_pair.collision_pair_by_shell:
    s_source = elliptical_pair.shell_index  # Source shell index (0-based)

    # Get species variable names including shell number
    species1_var_name = f'{species1_name}_{s_source + 1}'
    species2_var_name = f'{species2_name}_{s_source + 1}'

    # Get symbolic variables
    N_species1_s = species_sym_vars.get(species1_var_name)
    N_species2_s = species_sym_vars.get(species2_var_name)

    if N_species1_s is None or N_species2_s is None:
        continue  # Skip if species variables are not found

    phi_s = collision_pair.phi[s_source]
    nf_s = collision_pair.nf[s_source]

    # Compute the product of species populations (symbolic)
    product_sym = N_species1_s * N_species2_s

    # Losses of colliding species (gamma is -1)
    eq_loss = gamma[s_source] * phi_s * product_sym

    # Update 'eqs' for species1 and species2 at shell 's_source'
    idx_eqs_species1 = species_name_to_eqs_idx.get(species1_var_name)
    idx_eqs_species2 = species_name_to_eqs_idx.get(species2_var_name)

    eqs[s_source, idx_eqs_species1] += eq_loss
    eqs[s_source, idx_eqs_species2] += eq_loss

    # Process fragments from the collision
    fragments = elliptical_pair.fragments  # Should be an array of shape [n_destination_shells, n_mass_bins, n_ecc_bins]

    if fragments is not None:
        n_destination_shells, n_mass_bins, n_ecc_bins = fragments.shape

        for s_destination in range(n_destination_shells):
            fragments_sd = fragments[s_destination]  # Fragments ending up in shell s_destination
            for mass_bin_index in range(n_mass_bins):
                for ecc_bin_index in range(n_ecc_bins):
                    num_frags = fragments_sd[mass_bin_index, ecc_bin_index]
                    if num_frags != 0:
                        mass_value = mass_bins[mass_bin_index]
                        ecc_value = ecc_bins[ecc_bin_index]
                        # Generate debris species variable name including destination shell number
                        debris_species_name = f'N_{mass_value}_e{ecc_value}_{s_destination + 1}'
                        # Get index in 'eqs' for debris species
                        idx_eqs_debris = species_name_to_eqs_idx.get(debris_species_name)
                        if idx_eqs_debris is not None:
                            debris_var = species_sym_vars[debris_species_name]
                            # Compute delta gain for debris species (symbolic)
                            delta_gain = phi_s * product_sym * nf_s * num_frags
                            eqs[s_destination, idx_eqs_debris] += delta_gain
    else:
        continue  # If fragments are None, skip to the next elliptical_pair

# At this point, 'eqs' contains the symbolic equations for all species across all shells

# Print the 'eqs' matrix
print("Equations Matrix 'eqs' (n_shells x n_species):\n")
print(eqs)

Equations Matrix 'eqs' (n_shells x n_species):

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.000843835254011342*pi*B_e0.09375_4**2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00086369490730005*pi*B_e0.09375_2**2 + 0.000853678461171686*pi*B_e0.09375_3**2 + 0.00164930556292674*pi*B_e0.09375_6**2 + 0.00244591772292582*pi*B_e0.09375_7**2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.000806117048375066*pi*B_e0.09375_8**2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.000788199304968995*pi*B_e0.09375_10**2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00421917627005671*pi*B_e0.09375_4**2 + 0.00250248394121433*pi*B_e0.09375_5**2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00170735692234337*pi*B_e0.09375_3**2 + 0.00164930556292674*pi*B_e0.09375_6**2 + 0.00326122363056776*pi*B_e0.09375_7**2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.00087388868167737*pi*B_e0.09375_1**2 + 0.00086369490730005*pi*B_e0.09375_2**2 

In [195]:
import numpy as np
from sympy import symbols, Matrix, pi, S

scen_properties = elliptical
n_shells = scen_properties.n_shells

all_symbolic_vars = scen_properties.all_symbolic_vars

for collision_pair in scen_properties.collision_pairs:
    # map out the major components
    species1 = collision_pair.species1
    species2 = collision_pair.species2
    species1_name = species1.sym_name
    species2_name = species2.sym_name

    # Conversion factor
    meter_to_km = 1 / 1000

    # Compute collision cross-section (sigma)
    collision_pair.sigma = (species1.radius * meter_to_km + species2.radius * meter_to_km) ** 2

    # Compute collision rate (phi) for each shell
    collision_pair.phi = pi * scen_properties.v_imp2 / (scen_properties.V * meter_to_km**3) * collision_pair.sigma * S(86400) * S(365.25)

    # Determine if the collision is catastrophic for each shell
    catastrophic = is_catastrophic(species1.mass, species2.mass, scen_properties.v_imp2)

    # Compute the number of fragments (nf) for each shell
    nf = zeros(len(scen_properties.v_imp2), 1)
    for i, dv in enumerate(scen_properties.v_imp2):
        if catastrophic[i]:
            n_f_catastrophic = 0.1 * scen_properties.LC ** (-S(1.71)) * (species1.mass + species2.mass) ** (S(0.75))
            nf[i] = n_f_catastrophic
        else:
            n_f_damaging = 0.1 * scen_properties.LC ** (-S(1.71)) * (min(species1.mass, species2.mass) * dv ** 2) ** (S(0.75))
            nf[i] = n_f_damaging
    collision_pair.nf = nf.transpose()

    if isinstance(collision_pair.phi, (int, float, Expr)):
        phi_matrix = Matrix([collision_pair.phi] * len(collision_pair.gamma))
    else:
        phi_matrix = Matrix(collision_pair.phi)

    # Now we need to loop over each of the EllipticalCollisionPairs and create an overall gamma matrix
    for elliptical_pair in collision_pair.collision_pair_by_shell:
        s = elliptical_pair.shell_index

        # # Build full variable names including shell numbers
        # species1_var_name = f'{species1_name}_{s + 1}'  # Adjust for 1-based shell numbering
        # species2_var_name = f'{species2_name}_{s + 1}'

        # # Get indices for species1 and species2
        # species_and_shell_to_idx = {name: idx for idx, name in enumerate(scen_properties.all_symbolic_vars)}
        # sym_shell_idx_species1 = species_name_to_idx[species1_var_name]
        # sym_shell_idx_species2 = species_name_to_idx[species2_var_name]

        # print(f"Species 1: {species1_var_name}, Species 2: {species2_var_name}")
        # print(f"Index for Species 1: {idx_species1}, Index for Species 2: {idx_species2}")

        species_name_to_idx = {name: idx for idx, name in enumerate(scen_properties.species_names)}
        species1_idx = species_name_to_idx[species1.sym_name]
        species2_idx = species_name_to_idx[species2.sym_name]

        species_sym_vars = {str(var): var for var in all_symbolic_vars}

        print()

KeyboardInterrupt: 

In [185]:
scen_properties.collision_pairs[-1].collision_pair_by_shell[0].fragments

array([[[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.05, 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.05, 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.05, 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]],

       [[0.  , 0.  , 0.  , 0

In [199]:
scen_properties.all_symbolic_vars

[S_1,
 S_2,
 S_3,
 S_4,
 S_5,
 S_6,
 S_7,
 S_8,
 S_9,
 S_10,
 N_0.00141372kg_e0.00625_1,
 N_0.00141372kg_e0.00625_2,
 N_0.00141372kg_e0.00625_3,
 N_0.00141372kg_e0.00625_4,
 N_0.00141372kg_e0.00625_5,
 N_0.00141372kg_e0.00625_6,
 N_0.00141372kg_e0.00625_7,
 N_0.00141372kg_e0.00625_8,
 N_0.00141372kg_e0.00625_9,
 N_0.00141372kg_e0.00625_10,
 N_0.00141372kg_e0.018750000000000003_1,
 N_0.00141372kg_e0.018750000000000003_2,
 N_0.00141372kg_e0.018750000000000003_3,
 N_0.00141372kg_e0.018750000000000003_4,
 N_0.00141372kg_e0.018750000000000003_5,
 N_0.00141372kg_e0.018750000000000003_6,
 N_0.00141372kg_e0.018750000000000003_7,
 N_0.00141372kg_e0.018750000000000003_8,
 N_0.00141372kg_e0.018750000000000003_9,
 N_0.00141372kg_e0.018750000000000003_10,
 N_0.00141372kg_e0.043750000000000004_1,
 N_0.00141372kg_e0.043750000000000004_2,
 N_0.00141372kg_e0.043750000000000004_3,
 N_0.00141372kg_e0.043750000000000004_4,
 N_0.00141372kg_e0.043750000000000004_5,
 N_0.00141372kg_e0.043750000000000004_6,
 